In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

In [3]:
import mysql.connector
con = mysql.connector.connect(host='localhost', user='root', password='Pahomov2000+', db='nutritionaldb')

employees = pd.read_sql("SELECT * FROM employees", con)

In [4]:
data = pd.read_csv('E:\\Data Science\\Datasets\\EatingData\\result_table.csv')
data

,EmpId,Date,CmpxId,Dish_11,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_21,...,Dish_23,Dish_24,Dish_25,Dish_26,Dish_31,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36
0,9,2019-03-01,2,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
1,17,2019-03-01,2,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
2,28,2019-03-01,1,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
3,33,2019-03-01,2,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
4,35,2019-03-01,3,0,0,41,67,130,152,0,...,0,10,90,144,0,0,0,4,24,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85286,411,2020-09-18,1,0,0,47,52,167,171,0,...,0,19,54,109,0,0,0,35,145,164
85287,412,2020-09-18,3,0,0,47,52,167,171,0,...,0,19,54,109,0,0,0,35,145,164
85288,418,2020-09-18,2,0,0,47,52,167,171,0,...,0,19,54,109,0,0,0,35,145,164
85289,419,2020-09-18,4,0,0,47,52,167,171,0,...,0,19,54,109,0,0,0,35,145,164


In [5]:
data[data['Dish_31'] != 0]

,EmpId,Date,CmpxId,Dish_11,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_21,...,Dish_23,Dish_24,Dish_25,Dish_26,Dish_31,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36


In [6]:
remove_date = data[data['Dish_11'] != 0].Date.unique()[0]

In [7]:
data.drop(data[data['Date'] == remove_date].index, inplace=True)
data.reset_index(drop=True, inplace=True)

del data['Dish_11'], data['Dish_21'], data['Dish_31']

In [8]:
data

,EmpId,Date,CmpxId,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_22,Dish_23,Dish_24,Dish_25,Dish_26,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36
0,9,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152
1,17,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152
2,28,2019-03-01,1,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152
3,33,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152
4,35,2019-03-01,3,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85048,411,2020-09-18,1,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164
85049,412,2020-09-18,3,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164
85050,418,2020-09-18,2,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164
85051,419,2020-09-18,4,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164


In [9]:
# appending sex column to the table

sex = []
for emp in data.EmpId:
    if employees[employees['Id'] == emp].Employee.values[0][-1] == 'а':
        sex.append(0)
    else:
        sex.append(1)

data['Sex'] = sex
data

,EmpId,Date,CmpxId,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_22,Dish_23,Dish_24,Dish_25,Dish_26,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36,Sex
0,9,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,0
1,17,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,0
2,28,2019-03-01,1,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
3,33,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,0
4,35,2019-03-01,3,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85048,411,2020-09-18,1,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1
85049,412,2020-09-18,3,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,0
85050,418,2020-09-18,2,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,0
85051,419,2020-09-18,4,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1


In [10]:
data.groupby(['CmpxId']).size()

CmpxId
1    22907
2    28675
3    27906
4     5565
dtype: int64

In [11]:
data.drop(data[data['CmpxId'] == 4].index, inplace=True)
data

,EmpId,Date,CmpxId,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_22,Dish_23,Dish_24,Dish_25,Dish_26,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36,Sex
0,9,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,0
1,17,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,0
2,28,2019-03-01,1,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
3,33,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,0
4,35,2019-03-01,3,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85047,409,2020-09-18,1,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,0
85048,411,2020-09-18,1,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1
85049,412,2020-09-18,3,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,0
85050,418,2020-09-18,2,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,0


In [12]:
data.groupby(['CmpxId']).size()

CmpxId
1    22907
2    28675
3    27906
dtype: int64

In [13]:
list_of_rare_visitors = []

for emp in set(data['EmpId']):
    if data[data['EmpId'] == emp].shape[0] < 280:
        list_of_rare_visitors += data[data['EmpId'] == emp].index.tolist()

data.drop(list_of_rare_visitors, inplace=True)
data.reset_index(drop=True, inplace=True)

In [14]:
data

,EmpId,Date,CmpxId,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_22,Dish_23,Dish_24,Dish_25,Dish_26,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36,Sex
0,28,2019-03-01,1,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
1,46,2019-03-01,3,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
2,51,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
3,60,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
4,75,2019-03-01,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41508,402,2020-09-18,2,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1
41509,405,2020-09-18,1,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1
41510,408,2020-09-18,2,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1
41511,411,2020-09-18,1,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1


In [15]:
data.groupby(['CmpxId']).size()

CmpxId
1    12341
2    14054
3    15118
dtype: int64

In [16]:
del data['Date']

In [17]:
data

,EmpId,CmpxId,Dish_12,Dish_13,Dish_14,Dish_15,Dish_16,Dish_22,Dish_23,Dish_24,Dish_25,Dish_26,Dish_32,Dish_33,Dish_34,Dish_35,Dish_36,Sex
0,28,1,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
1,46,3,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
2,51,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
3,60,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
4,75,2,0,41,67,130,152,0,0,10,90,144,0,0,4,24,152,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41508,402,2,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1
41509,405,1,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1
41510,408,2,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1
41511,411,1,0,47,52,167,171,0,0,19,54,109,0,0,35,145,164,1


In [18]:
label = data.pop('CmpxId')

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data, label, shuffle=True, test_size=0.25, random_state=73)

In [20]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [21]:
model = xgb.XGBClassifier(base_score=0.51,
                              booster='gbtree',
                              max_depth=11,
                              subsample=1,
                              min_child_weight=1,
                              n_estimators=500,
                              learning_rate=0.3,
                              gamma=1.2,
                              objective='multi:softmax',
                              num_parallel_tree=2,
                              max_delta_step=1,
                              reg_alpha=0,
                              reg_lambda=1,
                              random_state=73)

In [22]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.51, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=1, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=2,
              objective='multi:softprob', random_state=73, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [23]:
pr = model.predict(X_test)

In [24]:
print('Accuracy: ' + str(round(accuracy_score(y_test, pr), 2) * 100) + '%')

Accuracy: 63.0%


In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pr)

array([[1794,  491,  798],
       [ 394, 2395,  736],
       [ 655,  753, 2363]], dtype=int64)

<p><b>I've got 63% accuracy...</b> Try to improve model later...</p>